<a href="https://colab.research.google.com/github/qazalmehrban/Medical-Segmentation/blob/ghazal/kera_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from skimage.transform import resize
import numpy as np
import nibabel as nib# for reading nifty files
import os 




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
IMG_HEIGHT, IMG_WIDTH =200,200

In [7]:
def gen():
  samples = '/content/drive/MyDrive/samples/'
  patients = next(os.walk(samples))[2]
  patients.sort()
  labels = '/content/drive/MyDrive/segmentation/'
  segmentations =next(os.walk(labels))[2]
  segmentations.sort()
  for i,j in zip(patients,segmentations):
    Images = nib.load((samples + i))
    im = np.array(Images.dataobj)
    Masks = nib.load((labels + j))    
    Masks = np.array(Masks.dataobj)
    img_resize = resize(im,
               (IMG_HEIGHT, IMG_WIDTH),
             mode = 'constant',
              anti_aliasing=True,
               preserve_range=True)
    mask_resize = resize(Masks,
               (IMG_HEIGHT, IMG_WIDTH),
             mode = 'constant',
              anti_aliasing=True,
               preserve_range=True)
    for k in range(im.shape[2]):
      im1 = np.expand_dims(img_resize[:,:,k],2)
      ma = np.expand_dims(mask_resize[:,:,k],2)

      yield {'Image':im1, 'Mask':ma}


In [8]:
dat = tf.data.Dataset.from_generator(gen, output_types ={"Image": tf.float32, "Mask": tf.float32}).batch(45)

In [9]:
it=iter(dat)

In [11]:
dataset = next(it)

In [13]:
dataset['Image']

<tf.Tensor: shape=(45, 200, 200, 1), dtype=float32, numpy=
array([[[[  15.434872 ],
         [  15.855372 ],
         [  14.938196 ],
         ...,
         [ 399.59338  ],
         [ 394.0894   ],
         [ 372.98026  ]],

        [[  28.613111 ],
         [  31.547318 ],
         [  35.31872  ],
         ...,
         [ 982.1221   ],
         [ 983.5691   ],
         [ 962.3449   ]],

        [[  19.82618  ],
         [  23.304947 ],
         [  26.99555  ],
         ...,
         [ 918.0546   ],
         [ 976.38403  ],
         [ 984.9792   ]],

        ...,

        [[  27.109987 ],
         [  24.816967 ],
         [  24.787155 ],
         ...,
         [1072.149    ],
         [1041.452    ],
         [ 883.1536   ]],

        [[  24.309105 ],
         [  19.806713 ],
         [  23.20557  ],
         ...,
         [1098.3776   ],
         [1093.6943   ],
         [ 907.1083   ]],

        [[  24.287306 ],
         [  33.618816 ],
         [  23.336592 ],
         ...,
        

In [14]:
dataset['Mask']

<tf.Tensor: shape=(45, 200, 200, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
   